In [125]:
import pyspark as ps
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.sql import Row
from pyspark.sql import functions as F
from pyspark.sql import DataFrameNaFunctions as DFna
from pyspark.sql.functions import udf, col, when
import matplotlib.pyplot as plt
import numpy as np
import math
import os

In [2]:
spark = ps.sql.SparkSession.builder \
            .master("local[4]") \
            .appName("building recommender") \
            .getOrCreate()
            
sc = spark.sparkContext
recs_df = spark.read.csv('data/instafreebie2-21-17_recommend_log.csv',
                     header=True,       # use headers or not
                     quote='"',         # char for quotes
                     sep=",",           # char for separation
                     inferSchema=True)  # do we infer schema or not ?
recs_df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- source: string (nullable = true)
 |-- reader_id: integer (nullable = true)
 |-- from_book_id: integer (nullable = true)
 |-- book_id: integer (nullable = true)
 |-- ad_id: integer (nullable = true)
 |-- boost_id: integer (nullable = true)
 |-- clicked: integer (nullable = true)
 |-- claimed: integer (nullable = true)
 |-- optin: integer (nullable = true)
 |-- created_at: timestamp (nullable = true)
 |-- updated_at: timestamp (nullable = true)



In [49]:
subset_df = spark.read.csv('data/subset.csv',
                     header=True,       # use headers or not
                     quote='"',         # char for quotes
                     sep=",",           # char for separation
                     inferSchema=True)  # do we infer schema or not ?
subset_df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- id: integer (nullable = true)
 |-- source: string (nullable = true)
 |-- reader_id: integer (nullable = true)
 |-- from_book_id: integer (nullable = true)
 |-- book_id: integer (nullable = true)
 |-- ad_id: integer (nullable = true)
 |-- boost_id: integer (nullable = true)
 |-- clicked: integer (nullable = true)
 |-- claimed: integer (nullable = true)
 |-- optin: integer (nullable = true)
 |-- created_at: timestamp (nullable = true)
 |-- updated_at: timestamp (nullable = true)



In [56]:
print subset_df.columns
type(subset_df)

['_c0', 'id', 'source', 'reader_id', 'from_book_id', 'book_id', 'ad_id', 'boost_id', 'clicked', 'claimed', 'optin', 'created_at', 'updated_at']


pyspark.sql.dataframe.DataFrame

In [57]:
subset_raw_data = sc.textFile('data/subset.csv')

In [58]:
type(subset_raw_data)

pyspark.rdd.RDD

In [59]:
subset_raw_data.take(3)

[u',id,source,reader_id,from_book_id,book_id,ad_id,boost_id,clicked,claimed,optin,created_at,updated_at',
 u'29295882,29295919,success_page,186643,20370,19203,0,0,1,1,0,2017-02-20 02:16:11,2017-02-20 02:26:24',
 u'29295883,29295920,success_page,186643,20370,19813,0,0,0,0,0,2017-02-20 02:16:11,2017-02-20 02:16:11']

In [61]:
subset_raw_data_header = subset_raw_data.take(1)[0]
subset_raw_data_header

u',id,source,reader_id,from_book_id,book_id,ad_id,boost_id,clicked,claimed,optin,created_at,updated_at'

In [74]:
subset_data = subset_raw_data.filter(lambda line: line!=subset_raw_data_header)\
    .map(lambda line: line.split(",")).map(lambda tokens: (tokens[3],tokens[5],tokens[9])).cache()

In [75]:
subset_data.take(4)

[(u'186643', u'19203', u'1'),
 (u'186643', u'19813', u'0'),
 (u'523754', u'16281', u'0'),
 (u'523754', u'22669', u'1')]

In [67]:
training_RDD, validation_RDD, test_RDD = subset_data.randomSplit([6, 2, 2], seed=0L)
validation_for_predict_RDD = validation_RDD.map(lambda x: (x[0], x[1]))
test_for_predict_RDD = test_RDD.map(lambda x: (x[0], x[1]))

In [65]:
training_df, validation_df, test_df = subset_df.randomSplit([.6, .2, .2], seed=0L)
type(training_df)

pyspark.sql.dataframe.DataFrame

In [40]:
iterations = 5
regularization_parameter = 0.1
ranks = [4, 6, 8, 10]
errors = []

In [45]:
min_error = float('inf')
best_rank = -1
best_iteration = -1

for rank in ranks:
    als = ALS(maxIter=iterations, regParam=regularization_parameter, rank=rank,\
              userCol="reader_id", itemCol="book_id", ratingCol="claimed", implicitPrefs=True)
    model = als.fit(training_RDD)
    predictions = model.transform(validation_RDD)
    new_predictions = predictions.filter(col('prediction') != np.nan)
    evaluator = RegressionEvaluator(metricName="rmse", labelCol="claimed",\
                                    predictionCol="prediction")
    rmse = evaluator.evaluate(new_predictions)
    errors.append(rmse)

    print 'For rank %s the RMSE is %s' % (rank, rmse)
    if rmse < min_error:
        min_error = rmse
        best_rank = rank
print 'The best model was trained with rank %s' % best_rank

For rank 4 the RMSE is 0.311012799435
For rank 6 the RMSE is 0.311002893167
For rank 8 the RMSE is 0.311081714229
For rank 10 the RMSE is 0.311152480803
The best model was trained with rank 6


In [123]:
from pyspark.mllib.recommendation import ALS as lib_ALS
seed = 5L
iterations = 10
regularization_parameter = 0.1
ranks = [4, 6, 8, 10, 12]
errors = [0, 0, 0, 0, 0]
err = 0
tolerance = 0.02

min_error = float('inf')
best_rank = -1
best_iteration = -1
for rank in ranks:
    model = lib_ALS.train(training_RDD, rank=rank, seed=seed, iterations=iterations,\
                      lambda_=regularization_parameter)
#     model = lib_ALS.trainImplicit(training_RDD, rank=rank, seed=seed, iterations=iterations,\
#                               lambda_=regularization_parameter)
    predictions = model.predictAll(validation_for_predict_RDD).map(lambda r: ((r[0], r[1]), r[2]))
    
#     als = ALS(maxIter=iterations, regParam=regularization_parameter, rank=rank,\
#               userCol="reader_id", itemCol="book_id", ratingCol="claimed", implicitPrefs=True)
#     model = als.fit(training)
    
    rates_and_preds = validation_RDD.map(lambda r: ((int(r[0]), int(r[1])), float(r[2]))).join(predictions)
    error = math.sqrt(rates_and_preds.map(lambda r: (r[1][0] - r[1][1])**2).mean())
    errors[err] = error
    err += 1
    print 'For rank %s the RMSE is %s' % (rank, error)
    if error < min_error:
        min_error = error
        best_rank = rank

print 'The best model was trained with rank %s' % best_rank

For rank 4 the RMSE is 0.289075543858
For rank 6 the RMSE is 0.289702572171
For rank 8 the RMSE is 0.288697953505
For rank 10 the RMSE is 0.288749551371
For rank 12 the RMSE is 0.289372235374
The best model was trained with rank 8


In [127]:
lines = spark.read.text("data/subset.csv").rdd
parts = lines.map(lambda row: row.value.split(","))
# subsetRDD = parts.map(lambda p: Row(reader_id=int(p[3]), book_id=int(p[5]), claimed=float(p[9])))
subsetRDD = parts.map(lambda p: Row(reader_id=int(p[3]), book_id=int(p[5]), claimed=int(p[9])))

In [128]:
subsetRDD.take(2)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 12586.0 failed 1 times, most recent failure: Lost task 0.0 in stage 12586.0 (TID 27207, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/Users/davidclausen/spark-2.1.0-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/worker.py", line 174, in main
    process()
  File "/Users/davidclausen/spark-2.1.0-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/worker.py", line 169, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/Users/davidclausen/spark-2.1.0-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/serializers.py", line 268, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/Users/davidclausen/spark-2.1.0-bin-hadoop2.7/python/pyspark/rdd.py", line 1339, in takeUpToNumLeft
    yield next(iterator)
  

In [129]:
ratings = spark.createDataFrame(subsetRDD)
(training, test) = ratings.randomSplit([0.8, 0.2])

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 12587.0 failed 1 times, most recent failure: Lost task 0.0 in stage 12587.0 (TID 27208, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/Users/davidclausen/spark-2.1.0-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/worker.py", line 174, in main
    process()
  File "/Users/davidclausen/spark-2.1.0-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/worker.py", line 169, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/Users/davidclausen/spark-2.1.0-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/serializers.py", line 268, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/Users/davidclausen/spark-2.1.0-bin-hadoop2.7/python/pyspark/rdd.py", line 1339, in takeUpToNumLeft
    yield next(iterator)
  

In [130]:
als = ALS(maxIter=5, regParam=0.01, userCol="reader_id", itemCol="book_id", ratingCol="claimed")
model = als.fit(training)

IllegalArgumentException: u'requirement failed: Column reader_id must be of type NumericType but was actually of type StringType.'